In [1]:
import numpy as np
import matplotlib.pyplot as plt
from outreading import read_F_D_edges, read_many_profiles
from utils import init_rate_matrix, construct_rate_matrix_from_F_D
from analyzeprofiles import extend_profile_water, extend_profile
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib import cm
from cauer import *
from scipy.integrate import quad, quadrature
import scipy
import matplotlib as mpl
%matplotlib qt

mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['figure.titlesize'] = 15
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20


def get_pbc_rate_matrix(F,D):
    n = len(F)
    rate = init_rate_matrix(n, F, D, pbc=True)
    return rate

def get_rate_matrix(F,D):
    n = len(F)
    rate = init_rate_matrix(n,F,D,pbc=True)
    # undo PBC
    rate[0,-1] = 0.
    rate[-1,0] = 0.
    # Constant A B BCs
    rate[0,0] = rate[0,1] = 0
    rate[-1,-1] = rate[-1,-2] = 0

    return rate

def get_Prate_matrix(F,D,P):
    n = len(F)
    rate = init_rate_matrix(n,F,D,pbc=True)
    rate[0,-1] = 0.
    rate[-1,0] = 0.
    # Constant A B BCs
    rate[0,0] = rate[1,0] = 0
    rate[-1,-1] = rate[-2,-1] = 0
    return rate

def propagate_with_R(time,R):
    prop = scipy.linalg.expm(time*R)
    init = np.zeros(len(prop))
    init[0] = 1.
    profile = np.dot(prop,init)
    return profile

def diagonalize_rate_matrix(R):
    eigval, eigvec = np.linalg.eig(R)
    # Sort from low to high, such that eigvec[:,-1] and eigvec[:,-2] are the lambda=0 eigvectors
    idx_sort = np.flip(eigval.argsort()[::-1])
    eigval = eigval[idx_sort]
    eigvec = eigvec[:,idx_sort]
    Diag = np.diag(eigval)
    Q = eigvec # R = Q.D.Q^-1, and thus exp(Rt) = Q.exp(Dt).Q^-1
    Qinv = np.linalg.inv(Q)
    max_diff = np.max(np.abs(np.matmul(np.matmul(Q,Diag),np.linalg.inv(Q))-R))
    print("||R - Q.D.Q^-1||_max = "+str(max_diff))
    return Diag,Q,Qinv

def propagate_with_diagonal(time,Diag,Q,Qinv):
    Diag_exp = np.zeros_like(Diag)
    for i in range(len(Diag)):
        Diag_exp[i,i] = np.exp(Diag[i,i]*time)
    prop = np.matmul(np.matmul(Q,Diag_exp),Qinv)
    init = np.zeros(len(prop))
    init[0]=1.
    profile = np.dot(prop,init)
    return profile

def get_eigenrates(R):
    eigval, eigvec = np.linalg.eig(R)
    # Sort from low to high, such that eigvec[:,-1] and eigvec[:,-2] are the lambda=0 eigvectors
    idx_sort = np.flip(eigval.argsort()[::-1])
    eigval = eigval[idx_sort]
    eigvec = eigvec[:,idx_sort]
    return eigval, eigvec

In [2]:
# Cutting the membrane part out of F and D profiles #
#####################################################

fn = "fit.popc.select.dat"
F,D,edges = read_F_D_edges(fn)
Drad = D # Not used, but just such that it has same dimensions.

nbins = len(F)
zpbc = edges[-1] - edges[0]
dz = (edges[-1]-edges[0])/(len(edges)-1.) #angstrom
dt = 1.

# BILAYER = bin [12,...,87] = 76 bins
# add 1 bin resembling water to the left and the right
# so 1 bin water + bilayer + 1 bin water = [11,...,88] = 78 bins
# And that is what we select, and define as 'short':
st = 11
end = 88

v_short = F[st:end+1]
d_short = D[st:end+1]
drad_short = Drad[st:end+1]
edges_short = edges[st:end+2] # has an extra element
z_array = np.array([i*dz for i in range(len(d_short))])

v_short[0] = F[0]
v_short[-1] = F[0]
d_short[0] = D[0]
d_short[-1] = D[0]


def get_properties(F, D, F_ref, D_ref, dz=1.):
    C = np.sum(np.exp(-(F-F_ref)))*dz 
    R = np.sum(1./(D*np.exp(-(F-F_ref))))*dz
    return C, R

print(get_properties(v_short[1:-1], d_short[1:-1], v_short[0], d_short[0], dz=dz))


(325.58738958637207, 385.42433560107196)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [3]:
R = get_rate_matrix(v_short,np.log(d_short/dz**2))
Diag,Q,Qinv = diagonalize_rate_matrix(R)
evals, evecs = get_eigenrates(R)
S_inf = np.sum(evecs[1:-1,-2]/evecs[0,-2])*dz
print("S_inf = "+str(S_inf))
inf_prof = evecs[:,-2]/evecs[0,-2]
profiles = []
times = []

for time in np.arange(0.1,1,0.001):
   times.append(time)
for time in np.arange(1,10,0.01):
   times.append(time)
for time in np.arange(10,100,0.1):
   times.append(time)
for time in np.arange(100,1000,1):
   times.append(time)
for time in np.arange(1000,10000,10):
   times.append(time)
for time in np.arange(10000,100000,100):
   times.append(time)
for time in np.arange(100000,1000000,100):  # for S(t) plot
# for time in np.arange(100000,1000000,1000):  # for c(t) evolution plot
   times.append(time)
times.append(1000000)
for i,time in enumerate(times):
   #profile = propagate_with_diagonal(time,Diag,Q,Qinv)
   profile = propagate_with_R(time,R)
   profiles.append(profile)
   if False:
       if (i+1)%int(len(times)/10)==0:
           print("did 10%")
profiles = np.array(profiles)
times = np.array(times)
times_ns = times/1000

||R - Q.D.Q^-1||_max = 4.02716597049102e-14
S_inf = 163.231611711113


In [4]:
times[-1]*10**-6

1.0

In [9]:
# Plotting the propagation of concentration in the membrane
# This cell is solely done for that.
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['figure.titlesize'] = 15
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20

colmap = cm.get_cmap("tab20c_r")

t0 = times[0]
t1 = times[-1]

L = np.log(t1)-np.log(t0)
t_shift = -1. * np.log(t0)

colors = (np.log(times)+t_shift)/L
#print("colors range from: ",colors[0], " to ", colors[-1])
#fig,ax =plt.subplots()
#ax.plot(colors)

fig,ax=plt.subplots()
for i,prof in enumerate(profiles[::-1]):
    if i == 0:
        pass
    if i%50==0:
        
    #ax.plot(z_array,prof,color=colmap(colors[i]))
    # color area under the curve
        ax.fill_between(z_array,0,prof,color=colmap(colors[::-1][i]),alpha=1)
ax.plot(z_array,inf_prof,color="k")
ax.set_xlabel(r"$z$ [$\mathrm{\AA}$]")
ax.set_ylabel(r"$c/c_{L}$", fontsize=25)
#ax.set_title(r"Propagation of oxygen concentration in POPC membrane")
ax.set_ylim(0,10.5)
ax.set_xlim(0,77*dz)
ax.axhline(1, color="k", linestyle="--")
#bbox_to_anchor=(xpos,ypos,xscale,yscale)
inaxes = inset_axes(ax,width="5%",height="70%",loc=1,bbox_to_anchor=(-0.001,-0.15,1,1),bbox_transform=ax.transAxes,borderpad=0)
inaxes.imshow(np.vstack((colors,colors)).T, cmap=colmap, aspect = 0.004)
tenfold_ids = [900*i for i in range(int(len(times)/900+1))]
ticktimes = [0.1*10**i for i in range(len(tenfold_ids))]
inaxes.set_yticklabels(["{:.0e}".format(tick/1000) for tick in ticktimes], fontsize=15)
inaxes.set_yticks(tenfold_ids)
inaxes.set_xticks([])
inaxes.set_title(r"time $t$ [ns]",fontsize=20, loc="right", pad=15)
ax.text(-0.25, 1.05, "B", transform=ax.transAxes, size=25, weight='bold')
fig.tight_layout()
fig.show()
fig.savefig("Propagation_one_POPC.pdf")


/home/wouter/anaconda/envs/openmm/lib/python3.7/site-packages/ipykernel_launcher.py:42: UserWarning: FixedFormatter should only be used together with FixedLocator
/home/wouter/anaconda/envs/openmm/lib/python3.7/site-packages/ipykernel_launcher.py:47: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [4]:
tau = evals[-3]**(-1)*-1
print("tau = "+str(tau))
# approximation of storage: S = S^inf * (1-exp(-t/tau))
S_approx = S_inf*(1-np.exp(-times/tau))
log_storages_approx = -np.log((S_inf - S_approx)/S_inf)

tau = 29811.784742690466


In [6]:
# Plotting the storage in the membrane, as a function of time
times_ns = times/1000
storages = []
for prof in profiles:
    storages.append(np.sum(prof[1:-1]*dz)) #do NOT include first and last bin, these are water
storages=np.array(storages)
log_storages = -np.log((S_inf - storages)/S_inf)

/home/wouter/anaconda/envs/openmm/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys


In [8]:
S_inf = 135.2402947125283

fig,ax=plt.subplots()
ax.plot(times_ns,storages/storages[-1]*S_inf,marker=".",color="red",label=r"$S(t)$", lw=1)
ax.plot(times_ns,S_approx/S_approx[-1]*S_inf,color="blue", ls=":", label=r"$S'(t)$", lw=3)
ax.axhline(S_inf, c="k", lw=2, ls="-", label=r"$S^\mathrm{ss}$")
ax.axhline(0.01*S_inf, c="k", label=r"[1, 10, 90, 99]% $S^\mathrm{ss}$",
           lw=1, ls="-.")
ax.axhline(0.1*S_inf,c="k",lw=2,ls="-.")
ax.axhline(0.9*S_inf,c="k",lw=2,ls="-.")
ax.axhline(0.99*S_inf,c="k",lw=2,ls="-.")
ax.plot(times_ns,storages/storages[-1]*S_inf,color="red",lw=2)
ax.plot(times_ns,S_approx/S_approx[-1]*S_inf,color="blue", ls="--", lw=2)

ax.tick_params(axis='both', which='major', labelsize=20)
ax.tick_params(axis='both', which='major', length=6)

ax.set_xlabel(r"$t$ [ns]")
ax.set_ylabel(r"$S(t)$ [#O$_2$/$\mu$m$^2$]")
#ax.set_title("Charging POPC membrane with oxygen")
ax.set_xlim(0,200)
ax.set_ylim(0,S_inf*1.1)
# add letter to panel 
ax.text(-0.25, 1.05, "C", transform=ax.transAxes, size=25, weight='bold')

ax.legend(loc="center right", fontsize=18)
fig.tight_layout()
#fig.savefig("charging_one_POPC_vs_time.png", dpi=300)
fig.savefig("charging_one_POPC_vs_time.pdf")

In [7]:
# Plotting the storage in the membrane, as a function of time
    
fig,ax=plt.subplots()
ax.plot(times_ns, 1-storages/S_inf, marker=".", color="red",label=r"$1-S(t)/S^{\mathrm{ss}}$", lw=1)
ax.plot(times_ns, 1-S_approx/S_inf, ls=":", color="blue",label=r"$1-S'(t)/S^{\mathrm{ss}}$", lw=3)
#ax.axhline(-np.log(0.99),c="k",label=r"[1, 10, 90, 99]% $S^\mathrm{SS}$",lw=2,ls="-.")
# ax.axhline(-np.log(0.9),c="k",lw=2,ls="-.")
# ax.axhline(-np.log(0.1),c="k",lw=2,ls="-.")
# ax.axhline(-np.log(0.01),c="k",lw=2,ls="-.")
ax.set_xlabel(r"$t$ [ns]")
ax.set_ylabel(r"$1-S(t)/S^{\mathrm{ss}}$"),
#ax.set_title("Charging POPC membrane with oxygen")
ax.set_xlim(0,200)
ax.set_ylim(1e-3,1)
# larger tickmarkings on x and y axes 
ax.tick_params(axis='both', which='major', labelsize=20)
# also the actual markings "-" 
ax.tick_params(axis='both', which='major', length=6)
ax.set_yscale("log")
ax.legend(loc="upper right", fontsize=18)
# add letter to panel
ax.text(-0.25, 1.05, "D", transform=ax.transAxes, size=25, weight='bold')
fig.tight_layout()
fig.savefig("charging_one_POPC_vs_time_log.pdf")

In [19]:
S_inf/0.45

163.231611711113

In [14]:
dz

0.67926

In [15]:
dz * 100

67.926

In [17]:
73.45e-22 / (32) * 6.022e23

138.22371875000002

In [20]:
S_10_10 = 2105.84475972119
S_1_10 = 578.6740036579552 #all in angstrom
S_0_10 = 163.2316117

#Due to linearity, we can subtract S_0_10 from S_10_10 to get the difference in storage:
S_diff = S_0_10
print("Difference of S/A/c* between pure water and 10 layers: "+str(S_diff)+" Angstrom")

c_air = 7. #mg/L @ 310K (37°C) and @ 159 mmHg pp_O2 (of the 760 mmHG pp_air)

#On the left we have 10 to 30 mmHg, and on the right we have 1-3 mmHG
#Let's take a difference of 10 mmHg

#Thus, we have 6.716 mg/L * 10 mmHg / 159 mmHg as c* 
c_prime = c_air * 10 / 159 #mg/L
print("c_prime: "+str(c_prime)+" mg/L")

molarmass = 32 #g/mol

c_prime_mol = c_prime * 10**(-3) * 10**(3) / 32 # mol/m^3

S_div_by_A = c_prime_mol * S_diff * 10**(-10) # mol/m^2

avog = 6.02214076 * 10**23 #entities/mole

A_SI = S_div_by_A * avog #entities/m^2

A = A_SI * 10**(-12)

print("amount of oxygen moleucles: "+str(A))


S_10_10 = 2105.84475972119
S_1_10 = 578.6740036579552 #all in angstrom
S_0_10 = 265.2

S_used = S_10_10

c_air = 7. #mg/L @ 310K (37°C) and @ 159 mmHg pp_O2 (of the 760 mmHG pp_air)

#On the left we have 10 to 30 mmHg, and on the right we have 1-3 mmHG
#Let's take a difference of 10 mmHg

#Thus, we have 6.716 mg/L * 10 mmHg / 159 mmHg as c*
c_prime = c_air * 10 / 159 #mg/L
print("c_prime: "+str(c_prime)+" mg/L")

molarmass = 32 #g/mol

c_prime_mol = c_prime * 10**(-3) * 10**(3) / 32 # mol/m^3

S_div_by_A = c_prime_mol * S_used * 10**(-10) # mol/m^2

avog = 6.02214076 * 10**23 #entities/mole

A_SI = S_div_by_A * avog #entities/m^2

A = A_SI * 10**(-12)

print("amount of oxygen moleucles: "+str(A))

Difference of S/A/c* between pure water and 10 layers: 163.2316117 Angstrom
c_prime: 0.44025157232704404 mg/L
amount of oxygen moleucles: 135.2402947125283
c_prime: 0.44025157232704404 mg/L
amount of oxygen moleucles: 1744.7298532280993
